In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Load models and data
with open('models/good_model.pkl', 'rb') as f:
    good_model = pickle.load(f)
with open('models/bad_model.pkl', 'rb') as f:
    bad_model = pickle.load(f)
with open('data/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

X_test = test_data['X_test']
y_test = test_data['y_test']

def test_model_robustness(model, X, y, n_trials=100):
    results = {
        'proxy_discrimination': test_proxies(model, X),
        'stability': test_stability(model, X),
        'cross_validation': cross_validate(model, X, y)
    }
    return results

def test_proxies(model, X):
    protected = ['persoon_geslacht_vrouw', 'persoon_leeftijd_bij_onderzoek']
    other_features = [c for c in X.columns if c not in protected]
    
    correlations = {}
    for p in protected:
        corrs = pd.Series({
            feat: abs(X[p].corr(X[feat]))
            for feat in other_features
        })
        correlations[p] = corrs.nlargest(3).to_dict()
    
    return correlations

def test_stability(model, X, noise_level=0.1):
    X_noisy = X + np.random.normal(0, noise_level, X.shape)
    
    original_preds = model.predict(X)
    noisy_preds = model.predict(X_noisy)
    
    return {
        'prediction_stability': np.mean(original_preds == noisy_preds),
        'output_variance': np.var(model.predict_proba(X_noisy), axis=0)
    }

def cross_validate(model, X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        scores.append(accuracy_score(y_val, y_pred))
    
    return {
        'mean_accuracy': np.mean(scores),
        'std_accuracy': np.std(scores)
    }

# Evaluate models
models = {'Good': good_model, 'Bad': bad_model}

for name, model in models.items():
    print(f"\n{name} Model Robustness Results:")
    results = test_model_robustness(model, X_test, y_test)
    
    print("\nProxy Discrimination:")
    for attr, proxies in results['proxy_discrimination'].items():
        print(f"\n{attr}:")
        for feat, corr in proxies.items():
            print(f"- {feat}: {corr:.3f}")
    
    print("\nStability:")
    print(f"Prediction stability: {results['stability']['prediction_stability']:.2%}")
    print(f"Output variance: {results['stability']['output_variance'].mean():.3f}")
    
    print("\nCross-validation:")
    print(f"Mean accuracy: {results['cross_validation']['mean_accuracy']:.3f}")
    print(f"Std accuracy: {results['cross_validation']['std_accuracy']:.3f}")